In [ ]:
from astroquery.sdss import SDSS
from astropy import coordinates as coords
import astropy.units as u
import requests
import pandas as pd
import os
import tqdm

SDSS.clear_cache()

In [ ]:
query_bands = ["u", "g", "r", "i", "z"]

In [ ]:
def download_data(CATALOG, parent_dir = "/Volumes/Backup Plus/Lohith_Files/Lensing_detection_data/SDSS_data/", sub_folder="spiral"):
    dest_path = os.path.join(parent_dir, sub_folder)
    if not os.path.exists(dest_path):
        os.makedirs(dest_path)

    QUERY_FAIL = []
    IMAGE_FAIL = []
    for i in tqdm.tqdm(range(0, len(CATALOG))):
        ra, dec = CATALOG.iloc[i]["ra"], CATALOG.iloc[i]["dec"]

        pos = coords.SkyCoord(ra, dec, unit="deg", frame="icrs")
        print(pos)
        result = SDSS.query_region(pos, radius=2* u.arcsec)

        #FITS
        if result:
            for b in query_bands:
                try:
                    imgs = SDSS.get_images(coordinates=pos, radius=100*u.arcsec, band=b, data_release=17)
                    imgs[0].writeto(os.path.join(dest_path, f"{b}_{i+1:06}.fits"), overwrite=True)
                except:
                    print("image not found :", i)
                    IMAGE_FAIL += [i]
        else:
            print("query fail :", i)
            QUERY_FAIL += [i]

        #JPEG
        url = f"https://skyserver.sdss.org/dr17/SkyServerWS/ImgCutout/getjpeg?ra={ra}&dec={dec}&scale=0.4&width=512&height=512"
        r = requests.get(url)
        open(os.path.join(dest_path, f"{i+1:06}.jpg"), "wb").write(r.content)   
    
    return QUERY_FAIL, IMAGE_FAIL


In [ ]:
spirals = pd.read_csv("./reference_csv/lens.csv")
temp = download_data(spirals, sub_folder="lens")